<a href="https://colab.research.google.com/github/nightitachi/NLP_Project-2024_US_Election_Sentiment_on_X/blob/main/2024_US_Election_Sentiment_on_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opendatasets

In [2]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/emirhanai/2024-u-s-election-sentiment-on-x')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alibelhrak
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/emirhanai/2024-u-s-election-sentiment-on-x


100%|██████████| 14.5k/14.5k [00:00<00:00, 20.2MB/s]

# Importing Libraries

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

# Loading Dataset

In [4]:
df_train = pd.read_csv('/content/2024-u-s-election-sentiment-on-x/train.csv')
df_val = pd.read_csv('/content/2024-u-s-election-sentiment-on-x/val.csv')
df_test = pd.read_csv('/content/2024-u-s-election-sentiment-on-x/test.csv')

In [5]:
df_train


,tweet_id,user_handle,timestamp,tweet_text,candidate,party,retweets,likes,sentiment
0,1,@user123,2024-11-03 08:45:00,Excited to see Kamala Harris leading the Democ...,Kamala Harris,Democratic Party,120,450,positive
1,2,@politicsFan,2024-11-03 09:15:23,Donald Trump's policies are the best for our e...,Donald Trump,Republican Party,85,300,positive
2,3,@greenAdvocate,2024-11-03 10:05:45,Jill Stein's environmental plans are exactly w...,Jill Stein,Green Party,60,200,positive
3,4,@indieVoice,2024-11-03 11:20:10,Robert Kennedy offers a fresh perspective outs...,Robert Kennedy,Independent,40,150,neutral
4,5,@libertyLover,2024-11-03 12:35:55,Chase Oliver's libertarian stance promotes tru...,Chase Oliver,Libertarian Party,30,120,positive
...,...,...,...,...,...,...,...,...,...
495,496,@user1067,2024-02-10 08:50:30,Kamala Harris is a symbol of progressive leade...,Kamala Harris,Democratic Party,95,360,positive
496,497,@econTalk7,2024-02-10 09:15:55,Trump's economic strategies are showing mixed ...,Donald Trump,Republican Party,100,440,neutral
497,498,@greenFutureNow7,2024-02-10 10:05:30,Jill Stein's solar projects are leading the wa...,Jill Stein,Green Party,75,240,positive
498,499,@indiePerspective7,2024-02-10 11:35:25,Robert Kennedy offers pragmatic solutions outs...,Robert Kennedy,Independent,18,85,neutral


In [6]:
df_train['candidate'].value_counts()

,count
candidate,
Kamala Harris,100
Donald Trump,100
Jill Stein,100
Robert Kennedy,100
Chase Oliver,100


In [7]:
df_train['sentiment'].value_counts()

,count
sentiment,
positive,328
neutral,126
negative,45
positive,1


# Data Preparation

In [8]:
df_train_text = df_train['tweet_text']
df_valid_text = df_val['tweet_text']
df_test_text = df_test['tweet_text']

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Tokenization
tokenizer = Tokenizer(oov_token="<oov>")
tokenizer.fit_on_texts(df_train_text)
tokenizer.fit_on_texts(df_valid_text)
tokenizer.fit_on_texts(df_test_text)

# Making Data into Sequences
train_sequences = tokenizer.texts_to_sequences(df_train_text)
valid_sequences = tokenizer.texts_to_sequences(df_valid_text)
test_sequences = tokenizer.texts_to_sequences(df_test_text)

# Making Pad_Sequences
max_seq_len = 50
train_pad_sequences = pad_sequences(train_sequences, maxlen=max_seq_len, padding='post')
valid_pad_sequences = pad_sequences(valid_sequences, maxlen=max_seq_len, padding='post')
test_pad_sequences = pad_sequences(test_sequences, maxlen=max_seq_len, padding='post')

df_train['input_sequences'] = list(train_pad_sequences)
df_val['input_sequences'] = list(valid_pad_sequences)
df_test['input_sequences'] = list(test_pad_sequences)

# Label Encoding
LA = LabelEncoder()
encoded_train_integer = LA.fit_transform(df_train['sentiment'])
encoded_valid_integer = LA.transform(df_val['sentiment'])
encoded_test_integer = LA.transform(df_test['sentiment'])

# Reshaping for OneHotEncoder
encoded_train_integer = encoded_train_integer.reshape(-1, 1)
encoded_valid_integer = encoded_valid_integer.reshape(-1, 1)
encoded_test_integer = encoded_test_integer.reshape(-1, 1)

# One-Hot Encoding
onehotencoder = OneHotEncoder(sparse_output=False)
train_onehot_encoder = onehotencoder.fit_transform(encoded_train_integer)
valid_onehot_encoder = onehotencoder.transform(encoded_valid_integer)
test_onehot_encoder = onehotencoder.transform(encoded_test_integer)

# Adding to DataFrames
df_train['sentiment_label'] = list(train_onehot_encoder)
df_val['sentiment_label'] = list(valid_onehot_encoder)
df_test['sentiment_label'] = list(test_onehot_encoder)

# Display DataFrames
print(df_train[['tweet_text', 'input_sequences', 'sentiment', 'sentiment_label']])
print(df_val[['tweet_text', 'input_sequences', 'sentiment', 'sentiment_label']])
print(df_test[['tweet_text', 'input_sequences', 'sentiment', 'sentiment_label']])


                                            tweet_text  \
0    Excited to see Kamala Harris leading the Democ...   
1    Donald Trump's policies are the best for our e...   
2    Jill Stein's environmental plans are exactly w...   
3    Robert Kennedy offers a fresh perspective outs...   
4    Chase Oliver's libertarian stance promotes tru...   
..                                                 ...   
495  Kamala Harris is a symbol of progressive leade...   
496  Trump's economic strategies are showing mixed ...   
497  Jill Stein's solar projects are leading the wa...   
498  Robert Kennedy offers pragmatic solutions outs...   
499  Chase Oliver is expanding his base among liber...   

                                       input_sequences sentiment  \
0    [343, 4, 344, 5, 15, 45, 10, 345, 199, 0, 0, 0...  positive   
1    [239, 6, 12, 3, 10, 346, 11, 286, 240, 0, 0, 0...  positive   
2    [7, 16, 181, 39, 3, 347, 348, 349, 182, 0, 0, ...  positive   
3    [8, 14, 57, 13, 217, 200, 

In [13]:
train_input = np.argmax(train_pad_sequences)
valid_input = np.argmax(valid_pad_sequences)
test_input = np.argmax(test_pad_sequences)

train_label = np.argmax(train_onehot_encoder)
valid_label = np.argmax(valid_onehot_encoder)
test_label = np.argmax(test_onehot_encoder)

# Creating The Model